In [2]:
from summarization_pipeline.pdf_section_extractor import extract_pdf_and_divide_sections
from summarization_pipeline.orchestration import Extractor
from summarization_pipeline.image_processing import extract_image_title_pairs, extract_titles_from_page, convert_response_to_list, get_important_image_paths

In [3]:
# Initializing the Extractor class for sending prompts to the LLaMA 2 70B model

extractor_70B_model = Extractor(model='70B')
extractor_13B_model = Extractor(model='13B')

In [4]:
# Getting and preprocessing PDF input

business_pdf1_path = "/Users/selinceydeli/Desktop/AIResearch/business-article-inputs/buss_article.pdf"
sections_dict = extract_pdf_and_divide_sections(business_pdf1_path)

In [4]:
# Extracting section texts of important sections 

abstract = sections_dict.get('abstract', "")
print("Abstract: " + abstract + "\n")

critical_sections = ["introduction", "conclusion", "discussion", "methodology"]

critical_section_information = {}
for section_name in critical_sections:
  critical_section_information[section_name] = sections_dict.get(section_name, "")

"""
If at least two of the sections among "conclusion", "discussion", and "outcomes" are missing, 
then take the last four sections (we keep each subsection seperately in the current formulation of sections_dict) 
of the article (excluding keywords, acknowledgments, and references sections)
"""
check_for_absence = ""
critical_section_list = list(critical_section_information.items())
for section_name, section_text in critical_section_list[-3:]:
    if section_text == "": check_for_absence += '0'

if len(check_for_absence) >= 2:
    accepted = 0
    unwanted_sections = ["keywords", "acknowledgments", "references"]
    sections_list = list(sections_dict.items())
    for section_name, section_text in sections_list[::-1]: # Reverse iteration of the sections_list
        section_name = section_name.lower()
        section_text = sections_dict.get(section_name, "")
        if section_name not in unwanted_sections and section_text != "":
            critical_section_information[section_name] = section_text
            accepted += 1
            if accepted >= 4:
                break

print("Important section titles:")
for key, value in critical_section_information.items():
    print(key)

Abstract:    
Business cycles (BCs) can alter the conditions for long-term business-to-business (B2B) relationships. Based on 
relationship marketing (RM) and interorganizational learning theories, the authors propose a model that explains 
relationship configurations that reveal opportunities under economic uncertainty. In the Pilot Study, the authors 
identify key mechanisms of RM process (communication openness, technical involvement, and customer value 
anticipation) and performance outcomes (price, cost-to-serve, and expectation of relationship continuity) from 
the supplier’s view. In Study 1, the proposed model is tested with a sample of large size, market leader firms in 
times of economic crisis (T1). In Study 2, conducting a multi-group analysis, the same sample is used to test the 
model in times of recovery/expansion (T2). The findings offer directions for suppliers on how to leverage B2B 
relationships through a BC. Particularly, the authors indicate that supplier’s perfor

In [4]:
# Summarizing important sections
    
summarized_sections = {}
for section_name, section_text in critical_section_information.items():
    if section_text != "": 
        summary = extractor_70B_model.summarize(section_name, section_text)
        summarized_sections[section_name] = summary
        print("Summary of " + section_name + ": \n" + summary)
    else : summarized_sections[section_name] = None

Summary of introduction: 
 This article discusses the importance of relationship marketing (RM) in business-to-business settings during economic contraction and expansion. It highlights the need for firms to adapt their RM strategies during economic fluctuations, citing previous research that has focused on general strategies for managing relationship performance. The authors argue that there is a lack of research integrating a business cycle (BC), an industrial setting, and an emerging economy, and aim to contribute to this body of research by studying the impact of RM process on determining the specific contribution of long-life customers in emerging economies during periods of contraction and expansion. They identify key relationship process mechanisms that help suppliers sustain or increase performance from their B2B long-life customers during economic swings, including communication openness, technical involvement, and customer value anticipation. The article concludes by emphasiz

In [5]:
# Hardcoded summarized sections

summarized_sections = '''
Summary of introduction: 
 This article discusses the importance of relationship marketing (RM) in business-to-business settings during economic contraction and expansion. It highlights the need for firms to adapt their RM strategies during economic fluctuations, citing previous research that has focused on general strategies for managing relationship performance. The authors argue that there is a lack of research integrating a business cycle (BC), an industrial setting, and an emerging economy, and aim to contribute to this body of research by studying the impact of RM process on determining the specific contribution of long-life customers in emerging economies during periods of contraction and expansion. They identify key relationship process mechanisms that help suppliers sustain or increase performance from their B2B long-life customers during economic swings, including communication openness, technical involvement, and customer value anticipation. The article concludes by emphasizing the timeliness and necessity of this study, especially in light of current crises such as the coronavirus situation and Ukraine conflict.
Summary of managerial implications: 
 This section discusses the managerial implications of the proposed Relationship Mechanism (RM) mechanisms for business-to-business (B2B) relationships during economic crises and recoveries. The authors suggest a 2x3 matrix with six quadrants, each representing a different combination of the three RM mechanisms: communication openness (COM), technical involvement (INV), and customer value anticipation (CVA). Each quadrant is named based on the empirical results and includes strategies for effectively managing each quadrant. The strategies include:

1. Value anticipation based on distant communication: Suppliers should reduce INV with customers while increasing CVA during economic crises. They can use communication media like email and telephone to establish high levels of communication without increasing technical collaboration.
2. Cost-oriented joint collaboration: During economic crises, suppliers should establish strong INV with customers while keeping ideas associated with customer value for future times. This can be achieved by concentrating technical work on cost-reducing opportunities and generating savings for both actors.
3. Dyadic top management consensus: To enhance expectation of continuity (EC) during economic crises, suppliers should exploit top management consensus with customers, nudging top managers to reach consensus on how to face the economic crisis together.
4. Generative hard work: During times of economic recovery, suppliers can increase PR by leveraging their ability to be technically involved with customers, having compatible top managers, and anticipating customer value altogether. INV is the most influential factor in this scenario.
5. Controlled technical deescalating: To reduce CTS during times of recovery, suppliers should carefully deescalate INV without affecting CVA or TMC.
6. Integrated optimal balance: During times of recovery, suppliers can integrate efforts on INV, CVA, and TMC to construct synergies and enhance EC. CVA and TMC are recommended as primary mechanisms since they do not have a negative impact on other goals during times of recovery.
Summary of theoretical implications: 
 This section discusses the theoretical implications of the study's findings on business-to-business (B2B) relationships during economic downturns. The authors argue that their findings have important implications for theory, as they provide insights into how B2B relationships can help firms navigate economic uncertainty. They extend previous research streams by showing that a BC brings profitability opportunities for B2B suppliers through nurturing mechanisms from long-term dyadic exchange. Additionally, they contribute to the "dark side" of B2B relationships' theoretical underpinnings by demonstrating how the inherent tension created in a BC can be managed through relationship management (RM) mechanisms. Finally, they note that their study contributes to extending BC marketing literature outside of the often-used US environment and provides insights into B2B relationships in emerging markets.
Summary of general discussion: 
 This research aims to provide guidance for managing an industrial business through a business cycle (BC) using marketing research, which has not been well-explored previously. The study focuses on the impact of price, cost-to-serve, and expectation of continuity variations during a BC in an emerging economy. Using a process approach with a temporal structure, the study finds that large size and market leader suppliers can survive and thrive through a BC by employing different relationship management mechanisms.
Summary of study  multigroup analysis after the crisis: 
 This article examines how different economic conditions affect the relevance of several relationship management mechanisms
'''

In [19]:
# Converting the section text information from dictionary to string
# to feed it to the model as input

def create_section_input(summarized_sections):
    # Initialize an empty string to store the formatted output
    section_input = ""

    # Iterate over each key-value pair in the dictionary
    for key, value in summarized_sections.items():
        # Append the key and value to the string with the specified format
        section_input += f"{key}: {value} \n"

    return section_input

section_input = create_section_input(summarized_sections)

In [12]:
# Enriching the abstract

enriched_abstract = extractor_70B_model.enrich_abstract(section_input, abstract)
print(enriched_abstract)

 This study explores the impact of business cycles (BCs) on long-term business-to-business (B2B) relationships and investigates the role of relationship marketing (RM) mechanisms in navigating economic uncertainty. Based on RM and interorganizational learning theories, the authors identify three RM mechanisms - communication openness, technical involvement, and customer value anticipation - and examine their effect on performance outcomes such as price, cost-to-serve, and expectation of relationship continuity.

The study consists of two parts. In Study 1, the authors test the proposed model during an economic crisis (T1) using a sample of large size, market leader firms. In Study 2, they conduct a multi-group analysis to test the model in times of recovery/expansion (T2) using the same sample. The findings provide guidance for suppliers on how to leverage B2B relationships through a BC, revealing that supplier performance is influenced differently by RM mechanisms during times of econ

In [ ]:
# Hardcoded enriched abstract

enriched_abstract = '''
This study explores the impact of business cycles (BCs) on long-term business-to-business (B2B) relationships and investigates the role of relationship marketing (RM) mechanisms in navigating economic uncertainty. Based on RM and interorganizational learning theories, the authors identify three RM mechanisms - communication openness, technical involvement, and customer value anticipation - and examine their effect on performance outcomes such as price, cost-to-serve, and expectation of relationship continuity.

The study consists of two parts. In Study 1, the authors test the proposed model during an economic crisis (T1) using a sample of large size, market leader firms. In Study 2, they conduct a multi-group analysis to test the model in times of recovery/expansion (T2) using the same sample. The findings provide guidance for suppliers on how to leverage B2B relationships through a BC, revealing that supplier performance is influenced differently by RM mechanisms during times of economic crisis versus times of recovery/expansion.

The study contributes to the theoretical underpinnings of B2B relationships, extending previous research streams by demonstrating how a BC brings profitability opportunities for B2B suppliers through nurturing mechanisms from long-term dyadic exchange. Additionally, it provides insights into B2B relationships in emerging markets and offers strategies for effectively managing six quadrants represented by different combinations of the three RM mechanisms.

Overall, the study highlights the importance of adapting RM strategies during economic fluctuations and offers valuable recommendations for businesses looking to sustain or increase their performance during periods of economic uncertainty.
'''

In [23]:
# Option 1: Extracting insights from the article using the summarized sections

section_input_abstract = "abstract: " + abstract + section_input
print(section_input_abstract + "\n")

insights = extractor_70B_model.extract_insights(section_input_abstract)
print("Extracted insights:\n" + insights)

abstract:    
Business cycles (BCs) can alter the conditions for long-term business-to-business (B2B) relationships. Based on 
relationship marketing (RM) and interorganizational learning theories, the authors propose a model that explains 
relationship configurations that reveal opportunities under economic uncertainty. In the Pilot Study, the authors 
identify key mechanisms of RM process (communication openness, technical involvement, and customer value 
anticipation) and performance outcomes (price, cost-to-serve, and expectation of relationship continuity) from 
the supplier’s view. In Study 1, the proposed model is tested with a sample of large size, market leader firms in 
times of economic crisis (T1). In Study 2, conducting a multi-group analysis, the same sample is used to test the 
model in times of recovery/expansion (T2). The findings offer directions for suppliers on how to leverage B2B 
relationships through a BC. Particularly, the authors indicate that supplier’s perfor

In [5]:
# Hardcoded insights extracted by the model

insights = '''
 Key Insights:

1. Business cycles (BCs) can impact long-term business-to-business (B2B) relationships, and understanding these effects can help firms navigate economic uncertainty.
2. Relationship marketing (RM) mechanisms can influence supplier performance differently during times of economic crisis versus times of recovery/expansion.
3. Six RM mechanism quadrants are proposed for effective management during economic crises and recoveries: value anticipation based on distant communication, cost-oriented joint collaboration, dyadic top management consensus, generative hard work, controlled technical deescalating, and integrated optimal balance.
4. The findings extend previous research streams by showing that a BC brings profitability opportunities for B2B suppliers through nurturing mechanisms from long-term dyadic exchange.
5. The study contributes to the "dark side" of B2B relationships' theoretical underpinnings by demonstrating how the inherent tension created in a BC can be managed through RM mechanisms.
6. The research provides insights into B2B relationships in emerging markets and extends BC marketing literature outside of the often-used US environment.
7. The study uses a process approach with a temporal structure to investigate the impact of price, cost-to-serve, and expectation of continuity variations during a BC in an emerging economy.
8. Large size and market leader suppliers can survive and thrive through a BC by employing different RM mechanisms.
'''

In [ ]:
# Option 2: Extracting insights from the article using the enriched abstract

insights = extractor_70B_model.extract_insights(enriched_abstract)
print("Extracted insights:\n" + insights)

In [7]:
# Generating a meaningful title to be presented as the chat title in the interface

title = extractor_13B_model.generate_title(insights)
print(title)

 Title: Navigating Economic Uncertainty in B2B Relationships: Leveraging Relationship Marketing Mechanisms During Business Cycles


In [ ]:
# Hardcoded title for the chat interface
# Title output by the 70B model

title = '''
 Navigating Economic Uncertainty: Leveraging Relationship Marketing Mechanisms to Strengthen Business-to-Business Relationships During Business Cycles
'''

In [ ]:
# Hardcoded title for the chat interface
# Title output by the 13B model

title = '''  
Title: Navigating Economic Uncertainty in B2B Relationships: Leveraging Relationship Marketing Mechanisms During Business Cycles.
'''

In [9]:
# Choosing the most important figures/tables of the article

import fitz

# Open the file
pdf_file = fitz.open(business_pdf1_path)
titles = []
image_title_pairs = {}
# Iterate over PDF pages
for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    page_image_title_pairs = extract_image_title_pairs(page,page_index)
    page_image_titles = extract_titles_from_page(page)
    image_title_pairs.update(page_image_title_pairs)
    for title in page_image_titles:
        title += " (Page:" + str(page_index+1) + ")"
        print(title)
        titles.append(title)

pdf_file.close()

image_titles = ""
for title in titles:
    image_titles += title + "\n"
    
important_images = extractor_13B_model.choose_images(insights, image_titles)
print(important_images)

Fig. 1. Overview of the Research Method (Page:4)
Fig. 2. Conceptual Model (Page:5)
Table 1. Sample Characteristics (Page:6)
Table 2. CFA Results (Page:7)
Table 3. Construct Correlations and AVEs (Page:7)
Table 4. MIIV-2SLS Results (Page:7)
Table 5. Measurement Invariance (Page:9)
Table 6. Results – Multi-group Analysis (from MLR estimation) (Page:9)
Table 7. Mechanisms for Successful Relationship Management of a Business Cycle (BC) (Page:9)
Fig. 3. Relationship Marketing (RM) Strategies Matrix (Page:11)


In [29]:
# Hardcoded important images (output of 70B model)

important_images = ''' 
 Based on the extracted insights, the three most important images of the article are:

1. Fig. 2. Conceptual Model (Page:5) - This image presents the conceptual model of the study, which illustrates the relationship between business cycles, relationship marketing mechanisms, and supplier performance. It provides a visual representation of the research question and the key variables under investigation.
2. Table 7. Mechanisms for Successful Relationship Management of a Business Cycle (BC) (Page:9) - This table presents the six RM mechanism quadrants that were identified in the study as being effective for managing B2B relationships during economic crises and recoveries. The table also includes examples of activities that fall within each quadrant, providing practical guidance for firms looking to navigate economic uncertainty.
3. Fig. 3. Relationship Marketing (RM) Strategies Matrix (Page:11) - This figure presents a matrix that combines the six RM mechanism quadrants with the three phases of a business cycle (pre-crisis, crisis, and post-crisis). The matrix provides a visual representation of how different RM strategies can be employed at different stages of an economic cycle, depending on the specific needs and challenges faced by suppliers. 
'''

In [5]:
# Hardcoded important images (output of 13B model)

important_images = '''
Sure! Based on the insights extracted from the article, here are the three most important images of the article along with their corresponding page numbers:

1. Fig. 2. Conceptual Model (Page: 5) - This image presents the conceptual model of the study, which highlights the relationship between business cycles, relationship marketing mechanisms, and firm performance. Understanding this model is crucial to grasping the key findings of the study.
2. Table 3. Construct Correlations and AVEs (Page: 7) - This table provides information on the correlations and average variance extracted (AVE) of the constructs used in the study. This table demonstrates the reliability and validity of the measures used, which is essential to understanding the study's results.
3. Fig. 3. Relationship Marketing (RM) Strategies Matrix (Page: 11) - This figure presents a matrix of relationship marketing strategies that can be employed by firms during different stages of the business cycle. This matrix provides a clear visual representation of the various RM strategies that can be effective in managing B2B relationships during economic uncertainty.
'''

In [10]:
# Displaying the fetched figures/tables that match the selected images

important_images_list = convert_response_to_list(important_images)

# Check whether the important image is extracted
found_images = get_important_image_paths(image_title_pairs, important_images_list)
print(found_images)

{'Fig. 3. Relationship Marketing (RM) Strategies Matrix': './images/page11/output_image0.png'}


In [ ]:
# Ending the trial session and logging the results to the logs.json file

extractor_13B_model.close()
extractor_70B_model.close()